In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, concatenate
import keras_tuner as kt

# Step 1: Define the Multi-Input, Multi-Output Model
def build_model(hp):
    # Hyperparameters for tuning
    units_1 = hp.Int('units_1', min_value=32, max_value=128, step=32)
    units_2 = hp.Int('units_2', min_value=32, max_value=128, step=32)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Input 1: Feature vector for regression task
    input_1 = Input(shape=(10,), name="input_1")  # Feature vector of length 10
    x1 = Dense(units_1, activation='relu')(input_1)
    x1 = Dense(units_2, activation='relu')(x1)

    # Input 2: Image data for classification task
    input_2 = Input(shape=(64, 64, 3), name="input_2")  # Example 64x64 RGB image
    x2 = Conv2D(32, (3, 3), activation='relu')(input_2)
    x2 = Conv2D(64, (3, 3), activation='relu')(x2)
    x2 = Flatten()(x2)

    # Combine both inputs
    combined = concatenate([x1, x2])

    # Output 1: Regression output
    output_1 = Dense(1, name="output_1")(combined)

    # Output 2: Classification output
    output_2 = Dense(10, activation='softmax', name="output_2")(combined)  # 10-class classification

    # Define the model
    model = Model(inputs=[input_1, input_2], outputs=[output_1, output_2])

    # Compile the model with the tunable learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss={'output_1': 'mse', 'output_2': 'categorical_crossentropy'},
        metrics={'output_1': 'mae', 'output_2': 'accuracy'}
    )
    return model

# Step 2: Set up Hyperparameter Tuning with Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of different hyperparameter sets to try
    executions_per_trial=2,  # Number of models built for each set of hyperparameters
    directory='my_dir',
    project_name='multi_input_multi_output_tuning'
)

# Generate random data for illustration purposes
x1_data = np.random.rand(1000, 10)  # Random feature vectors
x2_data = np.random.rand(1000, 64, 64, 3)  # Random image data
y1_data = np.random.rand(1000, 1)  # Random regression targets
y2_data = np.random.randint(0, 10, (1000, 1))  # Random classification targets (10 classes)

# One-hot encoding of the classification labels
y2_data = tf.keras.utils.to_categorical(y2_data, num_classes=10)

# Split data into training and validation sets
x1_train, x1_val = x1_data[:800], x1_data[800:]
x2_train, x2_val = x2_data[:800], x2_data[800:]
y1_train, y1_val = y1_data[:800], y1_data[800:]
y2_train, y2_val = y2_data[:800], y2_data[800:]

# Perform hyperparameter tuning
tuner.search(
    [x1_train, x2_train], 
    [y1_train, y2_train],
    validation_data=([x1_val, x2_val], [y1_val, y2_val]),
    epochs=5, 
    batch_size=32
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print a summary of the best model
best_model.summary()

# Step 3: Train the best model on the full dataset
best_model.fit(
    [x1_train, x2_train], 
    [y1_train, y2_train], 
    epochs=5, 
    batch_size=32
)


ModuleNotFoundError: No module named 'tensorflow'